In [1]:
import pandas as pd
import numpy as np
from forex_python.converter import CurrencyRates
from datetime import datetime
from pycbrf.toolbox import ExchangeRates
import warnings

warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.options.display.float_format= '{:,.2f}'.format

data = pd.read_excel('Initial data.csv')
data

,DT,DEAL_ID,BALMAIN,BALPRMN,PRODUCT,CURCODENUM,CURCODE,DOCBEGINDATE,ENDDATEWOPROLONG,ENDDATEWITHPROLONG,OVERDUETERMMAINDAY,OVERDUETERMPERCDAY,OVERDUEDATEMAIN,OVERDUEDATEPERC,DOCSUMCUR,DOCSUMRUR,MAINRESTCUR,MAINRESTRUR,PRMNRESTCUR,PRMNRESTRUR,DOCPROCRATE,SUMNACHPERCBALCUR,SUMNACHPERCBALRUR,SUMPRPERCBALCUR,SUMPRPERCBALRUR,RISKGROUP254,RESERVERATERPBU,SUMRESERVEMAINRPBU,SUMRESERVEPRMNRPBU,SUMRESERVEPRMN,SUMRESERVEPRPR
0,31.03.2020,6943416,nan,"45,815.00",Пенсионеры,810,Рубли,24.05.2011,24.05.2013,24.05.2013,"2,924.00","2,536.00",30.03.2012,22.04.2013,"41,000.00","41,000.00",nan,nan,"21,463.40","21,463.40",0.00,nan,nan,"4,523.44","4,523.44",5,100.00,nan,"21,463.40",nan,"4,523.44"
1,30.04.2020,6943416,nan,"45,815.00",Пенсионеры,810,Рубли,24.05.2011,24.05.2013,24.05.2013,"2,954.00","2,566.00",30.03.2012,22.04.2013,"41,000.00","41,000.00",nan,nan,"21,463.40","21,463.40",0.00,nan,nan,"4,523.44","4,523.44",5,100.00,nan,"21,463.40",nan,"4,523.44"
2,31.05.2020,6943416,nan,"45,815.00",Пенсионеры,810,Рубли,24.05.2011,24.05.2013,24.05.2013,"2,985.00","2,597.00",30.03.2012,22.04.2013,"41,000.00","41,000.00",nan,nan,"21,463.40","21,463.40",0.00,nan,nan,"4,523.44","4,523.44",5,100.00,nan,"21,463.40",nan,"4,523.44"
3,30.06.2020,6943416,nan,"45,815.00",Пенсионеры,810,Рубли,24.05.2011,24.05.2013,24.05.2013,"3,015.00","2,627.00",30.03.2012,22.04.2013,"41,000.00","41,000.00",nan,nan,"21,463.40","21,463.40",0.00,nan,nan,"4,523.44","4,523.44",5,100.00,nan,"21,463.40",nan,"4,523.44"
4,31.07.2020,6943416,nan,"45,815.00",Пенсионеры,810,Рубли,24.05.2011,24.05.2013,24.05.2013,"3,046.00","2,658.00",30.03.2012,22.04.2013,"41,000.00","41,000.00",nan,nan,"21,463.40","21,463.40",0.00,nan,nan,"4,523.44","4,523.44",5,100.00,nan,"21,463.40",nan,"4,523.44"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,30.04.2020,20757944533,nan,"45,815.00",Ипотека,810,Рубли,07.10.2019,07.10.2019,07.10.2019,207.00,207.00,07.10.2019,07.10.2019,"12,674,163.57","12,674,163.57",nan,nan,"3,798,757.08","3,798,757.08",0.00,nan,nan,"3,332,540.47","3,332,540.47",5,100.00,nan,"3,798,757.08",nan,"3,332,540.47"
6036,31.05.2020,20757944533,nan,"45,815.00",Ипотека,810,Рубли,07.10.2019,07.10.2019,07.10.2019,238.00,238.00,07.10.2019,07.10.2019,"12,674,163.57","12,674,163.57",nan,nan,"3,798,757.08","3,798,757.08",0.00,nan,nan,"3,332,540.47","3,332,540.47",5,100.00,nan,"3,798,757.08",nan,"3,332,540.47"
6037,30.06.2020,20757944533,nan,"45,815.00",Ипотека,810,Рубли,07.10.2019,07.10.2019,07.10.2019,268.00,268.00,07.10.2019,07.10.2019,"12,674,163.57","12,674,163.57",nan,nan,"3,798,757.08","3,798,757.08",0.00,nan,nan,"3,332,540.47","3,332,540.47",5,100.00,nan,"3,798,757.08",nan,"3,332,540.47"
6038,31.07.2020,20757944533,nan,"45,815.00",Ипотека,810,Рубли,07.10.2019,07.10.2019,07.10.2019,299.00,299.00,07.10.2019,07.10.2019,"12,674,163.57","12,674,163.57",nan,nan,"3,798,757.08","3,798,757.08",0.00,nan,nan,"3,332,540.47","3,332,540.47",5,100.00,nan,"3,798,757.08",nan,"3,332,540.47"


## Finding the exchange rate by parcing CB site

In [2]:
def get_currency(df, currency):
    if currency == 'RUB':
        df['CUR_RATE'] = 1
    elif currency == 'USD' or currency == 'EUR':
        df['CUR_RATE'] = df.apply(lambda x: float(str(ExchangeRates(x['DT'])[currency].rate).split()[0]), axis=1)
    return df

def count_provision(df):
    df['PROVISION'] = 0
    for column in ['SUMRESERVEMAINRPBU', 'SUMRESERVEPRMNRPBU', 'SUMRESERVEPRMN', 'SUMRESERVEPRPR']:
        df[column] = df[column].fillna(0)
        df['PROVISION'] += df[column]
        
    return df


def count_delta_provision(df):
    res_df = pd.DataFrame()
    credit_ids = df['DEAL_ID'].unique()
    for credit_id in credit_ids:
        tmp_df = df[df['DEAL_ID'] == credit_id]
        tmp_df = tmp_df.sort_values(by = ['DT'])
        tmp_df.reset_index().drop(columns = ['index'])
        deltas = [tmp_df.iloc[0]['PROVISION']]
        for i in range(1, tmp_df.shape[0]):
            prev_row = tmp_df.iloc[i-1]
            init_row = tmp_df.iloc[i]
            deltas.append(init_row['PROVISION'] - prev_row['PROVISION'])
        tmp_df['DELTA_PROVISION'] = deltas
        res_df = res_df.append(tmp_df)
    return res_df


def count_dA(df):
    res_df = pd.DataFrame()
    credit_ids = df['DEAL_ID'].unique()
    for credit_id in credit_ids:
        tmp_df = df[df['DEAL_ID'] == credit_id]
        tmp_df = tmp_df.sort_values(by = ['DT'])
        tmp_df.reset_index().drop(columns = ['index'])
        for column in ['MAINRESTCUR', 'PRMNRESTCUR']:
            tmp_df[column] = tmp_df[column].fillna(0)
        dA_s = [tmp_df.iloc[0]['MAINRESTCUR'] + tmp_df.iloc[0]['PRMNRESTCUR']]
        prev_value = dA_s[0]
        for i in range(1, tmp_df.shape[0]):
            prev_row = tmp_df.iloc[i-1]
            init_row = tmp_df.iloc[i]
            init_value = tmp_df.iloc[i]['MAINRESTCUR'] + tmp_df.iloc[i]['PRMNRESTCUR']
            dA_s.append(init_value - prev_value)
            prev_value = init_value
        tmp_df['dA'] = dA_s
        res_df = res_df.append(tmp_df)
    return res_df
        
def count_dC(df):
    res_df = pd.DataFrame()
    credit_ids = df['DEAL_ID'].unique()
    
    for credit_id in credit_ids:
        tmp_df = df[df['DEAL_ID'] == credit_id]
        tmp_df = tmp_df.sort_values(by = ['DT'])
        tmp_df.reset_index().drop(columns = ['index'])
        dC_s = [0]
        for i in range(1, tmp_df.shape[0]):
            prev_row = tmp_df.iloc[i-1]
            init_row = tmp_df.iloc[i]
            dC_s.append(init_row['CUR_RATE'] - prev_row['CUR_RATE'])
        tmp_df['dC'] = dC_s
        res_df = res_df.append(tmp_df)
    return res_df
        
    
def count_dR(df):
    res_df = pd.DataFrame()
    credit_ids = df['DEAL_ID'].unique()
    for credit_id in credit_ids:
        tmp_df = df[df['DEAL_ID'] == credit_id]
        tmp_df = tmp_df.sort_values(by = ['DT'])
        tmp_df.reset_index().drop(columns = ['index'])
        dR_s = [0]
        for i in range(1, tmp_df.shape[0]):
            prev_row = tmp_df.iloc[i-1]
            init_row = tmp_df.iloc[i]
            dR_s.append(init_row['RESERVERATERPBU'] - prev_row['RESERVERATERPBU'])
        tmp_df['dR'] = dR_s
        res_df = res_df.append(tmp_df)
    return res_df

def count_dPA(df):
    res_df = pd.DataFrame()
    credit_ids = df['DEAL_ID'].unique()
    for credit_id in credit_ids:
        tmp_df = df[df['DEAL_ID'] == credit_id]
        tmp_df = tmp_df.sort_values(by = ['DT'])
        tmp_df.reset_index().drop(columns = ['index'])
        
        dPA_s = [tmp_df.iloc[0]['dA'] * (tmp_df.iloc[0]['RESERVERATERPBU'] / 100) * tmp_df.iloc[0]['CUR_RATE']]
        for i in range(tmp_df.shape[0]-1):
            next_row = tmp_df.iloc[i+1]
            init_row = tmp_df.iloc[i]
            dPA_s.append(next_row['dA'] * (init_row['RESERVERATERPBU'] / 100) * init_row['CUR_RATE'])
            
        tmp_df['dPA'] = dPA_s
        res_df = res_df.append(tmp_df)
    return res_df

    
    df['dPA'] = df['dA'] * (df['RESERVERATERPBU'] / 100) * df['CUR_RATE']
    return df
    
        
def count_dPR(df):
    res_df = pd.DataFrame()
    credit_ids = df['DEAL_ID'].unique()
    for credit_id in credit_ids:
        tmp_df = df[df['DEAL_ID'] == credit_id]
        tmp_df = tmp_df.sort_values(by = ['DT'])
        tmp_df.reset_index().drop(columns = ['index'])
        dPR_s = [0]
        
        for column in ['MAINRESTCUR', 'PRMNRESTCUR']:
            tmp_df[column] = tmp_df[column].fillna(0)
            
        for i in range(1, tmp_df.shape[0]):
            init_row = tmp_df.iloc[i]
            dPR_s.append((init_row['MAINRESTCUR'] + init_row['PRMNRESTCUR']) * (init_row['dR'] / 100) * init_row['CUR_RATE'])
            
        tmp_df['dPR'] = dPR_s
        res_df = res_df.append(tmp_df)
    return res_df


def count_dPC(df):
    res_df = pd.DataFrame()
    credit_ids = df['DEAL_ID'].unique()
    for credit_id in credit_ids:
        tmp_df = df[df['DEAL_ID'] == credit_id]
        tmp_df = tmp_df.sort_values(by = ['DT'])
        tmp_df.reset_index().drop(columns = ['index'])
        dPC_s = [0]
        
        for column in ['RESERVERATERPBU', 'MAINRESTCUR', 'PRMNRESTCUR']:
            tmp_df[column] = tmp_df[column].fillna(0)
            
        for i in range(1, tmp_df.shape[0]):
            init_row = tmp_df.iloc[i]
            dPC_s.append(((init_row['MAINRESTCUR'] + init_row['PRMNRESTCUR']) * init_row['RESERVERATERPBU'] * init_row['dC']) / 100)
            
        tmp_df['dPC'] = dPC_s
        res_df = res_df.append(tmp_df)
    return res_df

def count_dXR(df):
    df['dXR'] = df['DELTA_PROVISION'].copy()
    for column in ['dPA', 'dPR', 'dPC']:
        df['dXR'] -= df[column]
    return df


def calculate_deltas(df):
    df = count_provision(df)
    df = count_delta_provision(df)
    df = count_dA(df)
    df = count_dC(df)
    df = count_dR(df)
    df = count_dPA(df)
    df = count_dPR(df)
    df = count_dPC(df)
    df = count_dXR(df)
    return df


In [3]:
cur_codes = {'RUB': 810,
             'EUR': 978,
             'USD': 840
            }

data['DT'] = pd.to_datetime(data['DT'])

rub_df = get_currency(data[data['CURCODENUM'] == cur_codes['RUB']], 'RUB')
usd_df = get_currency(data[data['CURCODENUM'] == cur_codes['USD']], 'USD')
eur_df = get_currency(data[data['CURCODENUM'] == cur_codes['EUR']], 'EUR')


df = rub_df.copy()
df = df.append(usd_df).append(eur_df)
df


,DT,DEAL_ID,BALMAIN,BALPRMN,PRODUCT,CURCODENUM,CURCODE,DOCBEGINDATE,ENDDATEWOPROLONG,ENDDATEWITHPROLONG,OVERDUETERMMAINDAY,OVERDUETERMPERCDAY,OVERDUEDATEMAIN,OVERDUEDATEPERC,DOCSUMCUR,DOCSUMRUR,MAINRESTCUR,MAINRESTRUR,PRMNRESTCUR,PRMNRESTRUR,DOCPROCRATE,SUMNACHPERCBALCUR,SUMNACHPERCBALRUR,SUMPRPERCBALCUR,SUMPRPERCBALRUR,RISKGROUP254,RESERVERATERPBU,SUMRESERVEMAINRPBU,SUMRESERVEPRMNRPBU,SUMRESERVEPRMN,SUMRESERVEPRPR,CUR_RATE
0,2020-03-31,6943416,nan,"45,815.00",Пенсионеры,810,Рубли,24.05.2011,24.05.2013,24.05.2013,"2,924.00","2,536.00",30.03.2012,22.04.2013,"41,000.00","41,000.00",nan,nan,"21,463.40","21,463.40",0.00,nan,nan,"4,523.44","4,523.44",5,100.00,nan,"21,463.40",nan,"4,523.44",1.00
1,2020-04-30,6943416,nan,"45,815.00",Пенсионеры,810,Рубли,24.05.2011,24.05.2013,24.05.2013,"2,954.00","2,566.00",30.03.2012,22.04.2013,"41,000.00","41,000.00",nan,nan,"21,463.40","21,463.40",0.00,nan,nan,"4,523.44","4,523.44",5,100.00,nan,"21,463.40",nan,"4,523.44",1.00
2,2020-05-31,6943416,nan,"45,815.00",Пенсионеры,810,Рубли,24.05.2011,24.05.2013,24.05.2013,"2,985.00","2,597.00",30.03.2012,22.04.2013,"41,000.00","41,000.00",nan,nan,"21,463.40","21,463.40",0.00,nan,nan,"4,523.44","4,523.44",5,100.00,nan,"21,463.40",nan,"4,523.44",1.00
3,2020-06-30,6943416,nan,"45,815.00",Пенсионеры,810,Рубли,24.05.2011,24.05.2013,24.05.2013,"3,015.00","2,627.00",30.03.2012,22.04.2013,"41,000.00","41,000.00",nan,nan,"21,463.40","21,463.40",0.00,nan,nan,"4,523.44","4,523.44",5,100.00,nan,"21,463.40",nan,"4,523.44",1.00
4,2020-07-31,6943416,nan,"45,815.00",Пенсионеры,810,Рубли,24.05.2011,24.05.2013,24.05.2013,"3,046.00","2,658.00",30.03.2012,22.04.2013,"41,000.00","41,000.00",nan,nan,"21,463.40","21,463.40",0.00,nan,nan,"4,523.44","4,523.44",5,100.00,nan,"21,463.40",nan,"4,523.44",1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2689,2020-04-30,63764213,nan,"45,815.00",Ипотека,978,Евро,24.01.2013,11.01.2016,11.01.2016,"1,982.00","2,160.00",27.11.2014,02.06.2014,"800,000.00","64,039,040.00",nan,nan,"261,932.39","20,967,373.50",12.00,nan,nan,"322,425.18","25,809,748.75",5,100.00,nan,"20,967,373.50",nan,"25,809,748.75",80.05
2690,2020-05-31,63764213,nan,"45,815.00",Ипотека,978,Евро,24.01.2013,11.01.2016,11.01.2016,"2,013.00","2,191.00",27.11.2014,02.06.2014,"800,000.00","62,839,120.00",nan,nan,"261,932.39","20,574,501.11",12.00,"2,662.27","209,118.38","322,425.18","25,326,143.22",5,100.00,nan,"20,574,501.11","209,118.38","25,326,143.22",78.55
2691,2020-06-30,63764213,nan,"45,815.00",Ипотека,978,Евро,24.01.2013,11.01.2016,11.01.2016,"2,043.00","2,221.00",27.11.2014,02.06.2014,"800,000.00","62,944,960.00",nan,nan,"261,932.39","20,609,154.76",12.00,nan,nan,"327,663.83","25,780,983.34",5,100.00,nan,"20,609,154.76",nan,"25,780,983.34",78.68
2692,2020-07-31,63764213,nan,"45,815.00",Ипотека,978,Евро,24.01.2013,11.01.2016,11.01.2016,"2,074.00","2,252.00",27.11.2014,02.06.2014,"800,000.00","69,002,560.00",nan,nan,"261,932.39","22,592,506.82",12.00,nan,nan,"330,326.09","28,491,682.31",5,100.00,nan,"22,592,506.82",nan,"28,491,682.31",86.25


## Breakdown by number of credits

In [4]:
grouped_by_deal_id = df[['DT','DEAL_ID']].groupby(by = ['DEAL_ID']).count()
grouped_by_deal_id = grouped_by_deal_id.reset_index()

credit_types = {}
credits_dfs = []

for i in range(1, 7):
    tmp_df = grouped_by_deal_id[grouped_by_deal_id['DT'] == i]
    credit_types[i] = tmp_df['DEAL_ID'].unique()
    
for i in range(1, 7):
    credits_dfs.append(df[df['DEAL_ID'].isin(credit_types[i])].reset_index().drop(columns = ['index']))
            

In [5]:
credits_dfs_new = []

### 1 credit 

In [6]:
cr_1 = calculate_deltas(credits_dfs[0])
credits_dfs_new.append(cr_1)
cr_1

,DT,DEAL_ID,BALMAIN,BALPRMN,PRODUCT,CURCODENUM,CURCODE,DOCBEGINDATE,ENDDATEWOPROLONG,ENDDATEWITHPROLONG,OVERDUETERMMAINDAY,OVERDUETERMPERCDAY,OVERDUEDATEMAIN,OVERDUEDATEPERC,DOCSUMCUR,DOCSUMRUR,MAINRESTCUR,MAINRESTRUR,PRMNRESTCUR,PRMNRESTRUR,DOCPROCRATE,SUMNACHPERCBALCUR,SUMNACHPERCBALRUR,SUMPRPERCBALCUR,SUMPRPERCBALRUR,RISKGROUP254,RESERVERATERPBU,SUMRESERVEMAINRPBU,SUMRESERVEPRMNRPBU,SUMRESERVEPRMN,SUMRESERVEPRPR,CUR_RATE,PROVISION,DELTA_PROVISION,dA,dC,dR,dPA,dPR,dPC,dXR
0,2020-03-31,62944933,"47,801.00",nan,Ипотека,810,Рубли,29.05.2013,29.05.2020,29.05.2020,nan,nan,NaN,NaN,"3,000,000.00","3,000,000.00","1,621.92","1,621.92",0.00,nan,10.50,nan,nan,nan,nan,2,0.35,5.68,0.00,0.00,0.00,1.00,5.68,5.68,"1,621.92",0,0,5.68,0,0,0.00
1,2020-03-31,63049299,"47,801.00",nan,Ипотека,810,Рубли,09.11.2006,08.11.2021,08.11.2021,nan,nan,NaN,NaN,"800,000.00","800,000.00","6,292.29","6,292.29",0.00,nan,14.00,nan,nan,nan,nan,2,0.35,22.02,0.00,0.00,0.00,1.00,22.02,22.02,"6,292.29",0,0,22.02,0,0,-0.00
2,2020-03-31,63060942,nan,nan,Потреб,810,Рубли,25.12.2013,23.12.2016,23.12.2016,nan,883.00,NaN,31.10.2017,"800,000.00","800,000.00",0.00,nan,0.00,nan,0.00,nan,nan,"2,931.81","2,931.81",5,100.00,0.00,0.00,0.00,"2,931.81",1.00,"2,931.81","2,931.81",0.00,0,0,0.00,0,0,"2,931.81"
3,2020-03-31,63245372,nan,"45,815.00",Ипотека,810,Рубли,21.02.2012,21.02.2037,21.02.2037,"1,896.00","1,828.00",22.01.2015,31.03.2015,"1,560,000.00","1,560,000.00",0.00,nan,"178,791.57","178,791.57",0.00,nan,nan,"759,222.16","759,222.16",5,100.00,0.00,"178,791.57",0.00,"759,222.16",1.00,"938,013.73","938,013.73","178,791.57",0,0,"178,791.57",0,0,"759,222.16"
4,2020-03-31,63246064,"45,507.00","45,815.00",Ипотека,810,Рубли,12.03.2014,12.03.2034,12.03.2034,1.00,1.00,31.03.2020,31.03.2020,"750,000.00","750,000.00","680,533.86","680,533.86","1,325.82","1,325.82",13.00,nan,nan,"7,432.00","7,432.00",2,1.50,"10,208.01",19.89,0.00,111.48,1.00,"10,339.38","10,339.38","681,859.68",0,0,"10,227.90",0,0,111.48
5,2020-03-31,63901471,"45,507.00",nan,Ипотека,810,Рубли,13.07.2012,13.07.2032,13.07.2032,nan,nan,NaN,NaN,"1,440,000.00","1,440,000.00","1,236,375.97","1,236,375.97",0.00,nan,13.00,nan,nan,nan,nan,2,0.50,"6,181.88",0.00,0.00,0.00,1.00,"6,181.88","6,181.88","1,236,375.97",0,0,"6,181.88",0,0,0.00
6,2020-03-31,63979552,"47,801.00",nan,Ипотека,810,Рубли,16.10.2013,16.10.2033,16.10.2033,nan,nan,NaN,NaN,"1,461,000.00","1,461,000.00","1,299,132.15","1,299,132.15",0.00,nan,12.75,nan,nan,nan,nan,2,0.35,"4,546.96",0.00,0.00,0.00,1.00,"4,546.96","4,546.96","1,299,132.15",0,0,"4,546.96",0,0,-0.00
7,2020-04-30,63992398,nan,nan,Потреб,810,Рубли,16.05.2013,16.05.2023,16.05.2023,1.00,1.00,30.04.2020,30.04.2020,"50,000,000.00","50,000,000.00",0.00,nan,0.00,nan,0.00,nan,nan,nan,nan,1,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0,0,0.00,0,0,0.00
8,2020-03-31,63997211,"47,801.00",nan,Ипотека,810,Рубли,11.12.2013,11.12.2028,11.12.2028,nan,nan,NaN,NaN,"3,080,000.00","3,080,000.00","2,409,062.60","2,409,062.60",0.00,nan,12.75,nan,nan,nan,nan,2,0.35,"8,431.72",0.00,0.00,0.00,1.00,"8,431.72","8,431.72","2,409,062.60",0,0,"8,431.72",0,0,0.00
9,2020-03-31,64032722,"45,507.00",nan,Ипотека,810,Рубли,25.07.2016,28.05.2028,28.05.2028,nan,nan,NaN,NaN,"7,307,812.05","7,307,812.05","2,847,936.60","2,847,936.60",0.00,nan,11.95,"107,551.20","107,551.20",nan,nan,3,50.00,"1,423,968.30",0.00,"53,775.60",0.00,1.00,"1,477,743.90","1,477,743.90","2,847,936.60",0,0,"1,423,968.30",0,0,"53,775.60"


### 2 credits 

In [7]:
cr_2 = calculate_deltas(credits_dfs[1])
credits_dfs_new.append(cr_2)
cr_2

,DT,DEAL_ID,BALMAIN,BALPRMN,PRODUCT,CURCODENUM,CURCODE,DOCBEGINDATE,ENDDATEWOPROLONG,ENDDATEWITHPROLONG,OVERDUETERMMAINDAY,OVERDUETERMPERCDAY,OVERDUEDATEMAIN,OVERDUEDATEPERC,DOCSUMCUR,DOCSUMRUR,MAINRESTCUR,MAINRESTRUR,PRMNRESTCUR,PRMNRESTRUR,DOCPROCRATE,SUMNACHPERCBALCUR,SUMNACHPERCBALRUR,SUMPRPERCBALCUR,SUMPRPERCBALRUR,RISKGROUP254,RESERVERATERPBU,SUMRESERVEMAINRPBU,SUMRESERVEPRMNRPBU,SUMRESERVEPRMN,SUMRESERVEPRPR,CUR_RATE,PROVISION,DELTA_PROVISION,dA,dC,dR,dPA,dPR,dPC,dXR
0,2020-03-31,28220635,nan,"45,815.00",Пенсионеры,810,Рубли,10.06.2014,20.06.2017,20.06.2017,"1,717.00","1,717.00",20.07.2015,20.07.2015,"100,000.00","100,000.00",0.00,nan,"73,983.68","73,983.68",0.00,nan,nan,"53,359.13","53,359.13",5,100.00,0.00,"73,983.68",0.00,"53,359.13",1.00,"127,342.81","127,342.81","73,983.68",0.00,0.00,"73,983.68",0.00,0.00,"53,359.13"
1,2020-04-30,28220635,nan,"45,815.00",Пенсионеры,810,Рубли,10.06.2014,20.06.2017,20.06.2017,"1,747.00","1,747.00",20.07.2015,20.07.2015,"100,000.00","100,000.00",0.00,nan,"73,983.68","73,983.68",0.00,nan,nan,"53,359.13","53,359.13",5,100.00,0.00,"73,983.68",0.00,"53,359.13",1.00,"127,342.81",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2020-06-30,62909499,nan,nan,Ипотека,810,Рубли,29.08.2012,29.08.2027,29.08.2027,1.00,1.00,30.06.2020,30.06.2020,"12,000,000.00","12,000,000.00",0.00,nan,0.00,nan,0.00,nan,nan,nan,nan,3,50.00,0.00,0.00,0.00,-0.01,1.00,-0.01,-0.01,0.00,0.00,0.00,0.00,0.00,0.00,-0.01
3,2020-08-31,62909499,nan,nan,Ипотека,810,Рубли,29.08.2012,29.08.2027,29.08.2027,1.00,1.00,31.08.2020,31.08.2020,"12,000,000.00","12,000,000.00",0.00,nan,0.00,nan,0.00,nan,nan,nan,nan,3,50.00,-0.01,-0.01,0.00,0.00,1.00,-0.02,-0.01,0.00,0.00,0.00,0.00,0.00,0.00,-0.01
4,2020-03-31,63043550,"47,801.00",nan,Ипотека,810,Рубли,29.10.2013,29.10.2038,29.10.2038,nan,nan,NaN,NaN,"2,380,000.00","2,380,000.00","112,397.33","112,397.33",0.00,nan,11.95,nan,nan,nan,nan,2,0.35,393.39,0.00,0.00,0.00,1.00,393.39,393.39,"112,397.33",0.00,0.00,393.39,0.00,0.00,-0.00
5,2020-04-30,63043550,"47,801.00",nan,Ипотека,810,Рубли,29.10.2013,29.10.2038,29.10.2038,nan,nan,NaN,NaN,"2,380,000.00","2,380,000.00","111,014.27","111,014.27",0.00,nan,11.95,nan,nan,nan,nan,2,0.35,388.55,0.00,0.00,0.00,1.00,388.55,-4.84,"-1,383.06",0.00,0.00,-4.84,0.00,0.00,0.00
6,2020-03-31,64106332,"47,801.00",nan,Ипотека,810,Рубли,04.10.2011,04.10.2026,04.10.2026,nan,nan,NaN,NaN,"4,000,000.00","4,000,000.00","950,243.56","950,243.56",0.00,nan,11.50,nan,nan,nan,nan,2,0.35,"3,325.85",0.00,0.00,0.00,1.00,"3,325.85","3,325.85","950,243.56",0.00,0.00,"3,325.85",0.00,0.00,-0.00
7,2020-04-30,64106332,"47,801.00",nan,Ипотека,810,Рубли,04.10.2011,04.10.2026,04.10.2026,nan,nan,NaN,NaN,"4,000,000.00","4,000,000.00","941,884.78","941,884.78",0.00,nan,11.50,nan,nan,nan,nan,2,0.35,"3,296.60",0.00,0.00,0.00,1.00,"3,296.60",-29.25,"-8,358.78",0.00,0.00,-29.26,0.00,0.00,0.01
8,2020-03-31,64219497,"47,801.00",nan,Ипотека,810,Рубли,19.03.2007,18.03.2022,18.03.2022,nan,nan,NaN,NaN,"900,000.00","900,000.00","220,432.23","220,432.23",0.00,nan,14.00,nan,nan,nan,nan,2,0.35,771.51,0.00,0.00,0.00,1.00,771.51,771.51,"220,432.23",0.00,0.00,771.51,0.00,0.00,-0.00
9,2020-04-30,64219497,"47,801.00",nan,Ипотека,810,Рубли,19.03.2007,18.03.2022,18.03.2022,nan,nan,NaN,NaN,"900,000.00","900,000.00","212,041.78","212,041.78",0.00,nan,14.00,nan,nan,nan,nan,2,0.35,742.15,0.00,0.00,0.00,1.00,742.15,-29.36,"-8,390.45",0.00,0.00,-29.37,0.00,0.00,0.01


### 3 credits 

In [8]:
cr_3 = calculate_deltas(credits_dfs[2])
credits_dfs_new.append(cr_3)
cr_3

,DT,DEAL_ID,BALMAIN,BALPRMN,PRODUCT,CURCODENUM,CURCODE,DOCBEGINDATE,ENDDATEWOPROLONG,ENDDATEWITHPROLONG,OVERDUETERMMAINDAY,OVERDUETERMPERCDAY,OVERDUEDATEMAIN,OVERDUEDATEPERC,DOCSUMCUR,DOCSUMRUR,MAINRESTCUR,MAINRESTRUR,PRMNRESTCUR,PRMNRESTRUR,DOCPROCRATE,SUMNACHPERCBALCUR,SUMNACHPERCBALRUR,SUMPRPERCBALCUR,SUMPRPERCBALRUR,RISKGROUP254,RESERVERATERPBU,SUMRESERVEMAINRPBU,SUMRESERVEPRMNRPBU,SUMRESERVEPRMN,SUMRESERVEPRPR,CUR_RATE,PROVISION,DELTA_PROVISION,dA,dC,dR,dPA,dPR,dPC,dXR
0,2020-03-31,62873186,"47,801.00",nan,Ипотека,810,Рубли,14.12.2012,14.12.2032,14.12.2032,nan,nan,NaN,NaN,"2,500,000.00","2,500,000.00","1,174,571.63","1,174,571.63",0.00,nan,13.00,nan,nan,nan,nan,2,0.35,"4,111.00",0.00,0.00,0.00,1.00,"4,111.00","4,111.00","1,174,571.63",0.00,0.00,"4,111.00",0.00,0.00,-0.00
1,2020-04-30,62873186,"47,801.00",nan,Ипотека,810,Рубли,14.12.2012,14.12.2032,14.12.2032,nan,nan,NaN,NaN,"2,500,000.00","2,500,000.00","1,157,746.55","1,157,746.55",0.00,nan,13.00,nan,nan,nan,nan,2,0.35,"4,052.11",0.00,0.00,0.00,1.00,"4,052.11",-58.89,"-16,825.08",0.00,0.00,-58.89,0.00,0.00,-0.00
2,2020-05-31,62873186,"47,801.00",nan,Ипотека,810,Рубли,14.12.2012,14.12.2032,14.12.2032,nan,nan,NaN,NaN,"2,500,000.00","2,500,000.00","1,157,746.55","1,157,746.55",0.00,nan,13.00,"12,747.87","12,747.87",nan,nan,2,0.35,"4,052.11",0.00,44.62,0.00,1.00,"4,096.73",44.62,0.00,0.00,0.00,0.00,0.00,0.00,44.62
3,2020-03-31,62995490,"47,801.00","45,815.00",Ипотека,810,Рубли,26.09.2011,31.08.2026,31.08.2026,1.00,1.00,31.03.2020,31.03.2020,"2,501,746.10","2,501,746.10","849,312.39","849,312.39","7,944.58","7,944.58",10.75,nan,nan,"7,283.95","7,283.95",2,1.50,"12,739.69",119.17,0.00,109.26,1.00,"12,968.12","12,968.12","857,256.97",0.00,0.00,"12,858.85",0.00,0.00,109.27
4,2020-04-30,62995490,"47,801.00","45,815.00",Ипотека,810,Рубли,26.09.2011,31.08.2026,31.08.2026,1.00,1.00,30.04.2020,30.04.2020,"2,501,746.10","2,501,746.10","841,041.08","841,041.08","8,271.31","8,271.31",10.75,nan,nan,"7,212.22","7,212.22",2,1.50,"12,615.62",124.07,0.00,108.18,1.00,"12,847.87",-120.25,"-7,944.58",0.00,0.00,-119.17,0.00,0.00,-1.08
5,2020-05-31,62995490,"47,801.00",nan,Ипотека,810,Рубли,26.09.2011,31.08.2026,31.08.2026,nan,nan,NaN,NaN,"2,501,746.10","2,501,746.10","841,041.08","841,041.08",0.00,nan,10.75,"7,657.84","7,657.84",nan,nan,2,0.35,"2,943.64",0.00,26.80,0.00,1.00,"2,970.44","-9,877.43","-8,271.31",0.00,-1.15,-124.07,"-9,671.97",0.00,-81.39
6,2020-03-31,63093260,"47,801.00",nan,Ипотека,810,Рубли,22.08.2012,22.08.2020,22.08.2020,nan,nan,NaN,NaN,"4,834,000.00","4,834,000.00","235,002.53","235,002.53",0.00,nan,12.25,nan,nan,nan,nan,2,0.35,822.51,0.00,0.00,0.00,1.00,822.51,822.51,"235,002.53",0.00,0.00,822.51,0.00,0.00,0.00
7,2020-04-30,63093260,"47,801.00",nan,Ипотека,810,Рубли,22.08.2012,22.08.2020,22.08.2020,nan,nan,NaN,NaN,"4,834,000.00","4,834,000.00","157,133.19","157,133.19",0.00,nan,12.25,nan,nan,nan,nan,2,0.35,549.97,0.00,0.00,0.00,1.00,549.97,-272.54,"-77,869.34",0.00,0.00,-272.54,0.00,0.00,0.00
8,2020-05-31,63093260,"47,801.00",nan,Ипотека,810,Рубли,22.08.2012,22.08.2020,22.08.2020,nan,nan,NaN,NaN,"4,834,000.00","4,834,000.00","157,133.19","157,133.19",0.00,nan,12.25,"1,630.36","1,630.36",nan,nan,2,0.35,549.97,0.00,5.71,0.00,1.00,555.68,5.71,0.00,0.00,0.00,0.00,0.00,0.00,5.71
9,2020-03-31,63414135,"47,801.00",nan,Ипотека,810,Рубли,08.08.2013,08.08.2033,08.08.2033,nan,nan,NaN,NaN,"1,408,000.00","1,408,000.00","808,767.07","808,767.07",0.00,nan,13.25,nan,nan,nan,nan,2,0.35,"2,830.68",0.00,0.00,0.00,1.00,"2,830.68","2,830.68","808,767.07",0.00,0.00,"2,830.68",0.00,0.00,-0.00


### 4 credits 

In [9]:
cr_4 = calculate_deltas(credits_dfs[3])
credits_dfs_new.append(cr_4)
cr_4.head(60)

,DT,DEAL_ID,BALMAIN,BALPRMN,PRODUCT,CURCODENUM,CURCODE,DOCBEGINDATE,ENDDATEWOPROLONG,ENDDATEWITHPROLONG,OVERDUETERMMAINDAY,OVERDUETERMPERCDAY,OVERDUEDATEMAIN,OVERDUEDATEPERC,DOCSUMCUR,DOCSUMRUR,MAINRESTCUR,MAINRESTRUR,PRMNRESTCUR,PRMNRESTRUR,DOCPROCRATE,SUMNACHPERCBALCUR,SUMNACHPERCBALRUR,SUMPRPERCBALCUR,SUMPRPERCBALRUR,RISKGROUP254,RESERVERATERPBU,SUMRESERVEMAINRPBU,SUMRESERVEPRMNRPBU,SUMRESERVEPRMN,SUMRESERVEPRPR,CUR_RATE,PROVISION,DELTA_PROVISION,dA,dC,dR,dPA,dPR,dPC,dXR
0,2020-03-31,62948423,"47,801.00","45,815.00",Ипотека,810,Рубли,06.03.2012,28.02.2022,28.02.2022,1.00,nan,31.03.2020,NaN,"1,218,617.96","1,218,617.96","91,474.56","91,474.56","3,319.27","3,319.27",17.00,nan,nan,nan,nan,2,1.50,"1,372.12",49.79,0.00,0.00,1.00,"1,421.91","1,421.91","94,793.83",0.00,0.00,"1,421.91",0.00,0.00,0.00
1,2020-04-30,62948423,"47,801.00","45,815.00",Ипотека,810,Рубли,06.03.2012,28.02.2022,28.02.2022,1.00,1.00,30.04.2020,30.04.2020,"1,218,617.96","1,218,617.96","88,060.21","88,060.21","3,414.35","3,414.35",17.00,nan,nan,593.92,593.92,2,1.50,"1,320.90",51.22,0.00,8.91,1.00,"1,381.03",-40.88,"-3,319.27",0.00,0.00,-49.79,0.00,0.00,8.91
2,2020-05-31,62948423,"47,801.00",nan,Ипотека,810,Рубли,06.03.2012,28.02.2022,28.02.2022,nan,nan,NaN,NaN,"1,218,617.96","1,218,617.96","88,060.21","88,060.21",0.00,nan,17.00,"1,267.97","1,267.97",nan,nan,2,0.35,308.21,0.00,4.44,0.00,1.00,312.65,"-1,068.38","-3,414.35",0.00,-1.15,-51.22,"-1,012.69",0.00,-4.47
3,2020-06-30,62948423,"47,801.00","45,815.00",Ипотека,810,Рубли,06.03.2012,28.02.2022,28.02.2022,1.00,1.00,30.06.2020,30.06.2020,"1,218,617.96","1,218,617.96","81,131.17","81,131.17","3,508.01","3,508.01",17.00,nan,nan,845.77,845.77,2,1.50,"1,216.97",52.62,0.00,12.69,1.00,"1,282.28",969.63,"-3,421.03",0.00,1.15,-11.97,973.35,0.00,8.25
4,2020-03-31,63173417,nan,"45,817.00",Ипотека,810,Рубли,30.10.2013,30.10.2033,30.10.2033,"1,645.00","1,584.00",30.09.2015,30.11.2015,"2,723,000.00","2,723,000.00",0.00,nan,"2,276,452.88","2,276,452.88",0.00,nan,nan,"437,346.20","437,346.20",5,100.00,0.00,"2,276,452.88",0.00,"437,346.20",1.00,"2,713,799.08","2,713,799.08","2,276,452.88",0.00,0.00,"2,276,452.88",0.00,0.00,"437,346.20"
5,2020-04-30,63173417,nan,"45,817.00",Ипотека,810,Рубли,30.10.2013,30.10.2033,30.10.2033,"1,675.00","1,614.00",30.09.2015,30.11.2015,"2,723,000.00","2,723,000.00",0.00,nan,"2,276,452.88","2,276,452.88",0.00,nan,nan,"437,346.20","437,346.20",5,100.00,0.00,"2,276,452.88",0.00,"437,346.20",1.00,"2,713,799.08",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,2020-05-31,63173417,nan,"45,817.00",Ипотека,810,Рубли,30.10.2013,30.10.2033,30.10.2033,"1,706.00","1,645.00",30.09.2015,30.11.2015,"2,723,000.00","2,723,000.00",0.00,nan,"2,276,452.88","2,276,452.88",0.00,nan,nan,"437,346.20","437,346.20",5,100.00,0.00,"2,276,452.88",0.00,"437,346.20",1.00,"2,713,799.08",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,2020-06-30,63173417,nan,"45,817.00",Ипотека,810,Рубли,30.10.2013,30.10.2033,30.10.2033,"1,736.00","1,675.00",30.09.2015,30.11.2015,"2,723,000.00","2,723,000.00",0.00,nan,"2,276,452.88","2,276,452.88",0.00,nan,nan,"437,346.20","437,346.20",5,100.00,0.00,"2,276,452.88",0.00,"437,346.20",1.00,"2,713,799.08",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,2020-03-31,63242168,"47,801.00",nan,Ипотека,810,Рубли,14.09.2012,14.09.2027,14.09.2027,nan,nan,NaN,NaN,"2,560,000.00","2,560,000.00","296,650.49","296,650.49",0.00,nan,11.75,nan,nan,nan,nan,2,0.35,"1,038.28",0.00,0.00,0.00,1.00,"1,038.28","1,038.28","296,650.49",0.00,0.00,"1,038.28",0.00,0.00,0.00
9,2020-04-30,63242168,"47,801.00",nan,Ипотека,810,Рубли,14.09.2012,14.09.2027,14.09.2027,nan,nan,NaN,NaN,"2,560,000.00","2,560,000.00","138,841.37","138,841.37",0.00,nan,11.75,nan,nan,nan,nan,2,0.35,485.94,0.00,0.00,0.00,1.00,485.94,-552.34,"-157,809.12",0.00,0.00,-552.33,0.00,0.00,-0.01


### 5 credits 

In [10]:
cr_5 = calculate_deltas(credits_dfs[4])
credits_dfs_new.append(cr_5)
cr_5

,DT,DEAL_ID,BALMAIN,BALPRMN,PRODUCT,CURCODENUM,CURCODE,DOCBEGINDATE,ENDDATEWOPROLONG,ENDDATEWITHPROLONG,OVERDUETERMMAINDAY,OVERDUETERMPERCDAY,OVERDUEDATEMAIN,OVERDUEDATEPERC,DOCSUMCUR,DOCSUMRUR,MAINRESTCUR,MAINRESTRUR,PRMNRESTCUR,PRMNRESTRUR,DOCPROCRATE,SUMNACHPERCBALCUR,SUMNACHPERCBALRUR,SUMPRPERCBALCUR,SUMPRPERCBALRUR,RISKGROUP254,RESERVERATERPBU,SUMRESERVEMAINRPBU,SUMRESERVEPRMNRPBU,SUMRESERVEPRMN,SUMRESERVEPRPR,CUR_RATE,PROVISION,DELTA_PROVISION,dA,dC,dR,dPA,dPR,dPC,dXR
0,2020-03-31,62837355,"45,507.00",nan,Ипотека,810,Рубли,24.05.2012,24.05.2022,24.05.2022,nan,nan,NaN,NaN,"700,000.00","700,000.00","227,623.65","227,623.65",0.00,nan,12.50,nan,nan,nan,nan,2,0.50,"1,138.12",0.00,0.00,0.00,1.00,"1,138.12","1,138.12","227,623.65",0.00,0.00,"1,138.12",0.00,0.00,0.00
1,2020-04-30,62837355,"45,507.00",nan,Ипотека,810,Рубли,24.05.2012,24.05.2022,24.05.2022,nan,nan,NaN,NaN,"700,000.00","700,000.00","219,576.86","219,576.86",0.00,nan,12.50,nan,nan,nan,nan,2,0.50,"1,097.88",0.00,0.00,0.00,1.00,"1,097.88",-40.24,"-8,046.79",0.00,0.00,-40.23,0.00,0.00,-0.01
2,2020-05-31,62837355,"45,507.00",nan,Ипотека,810,Рубли,24.05.2012,24.05.2022,24.05.2022,nan,nan,NaN,NaN,"700,000.00","700,000.00","219,576.86","219,576.86",0.00,nan,12.50,"2,324.76","2,324.76",nan,nan,2,0.50,"1,097.88",0.00,11.62,0.00,1.00,"1,109.50",11.62,0.00,0.00,0.00,0.00,0.00,0.00,11.62
3,2020-06-30,62837355,"45,507.00",nan,Ипотека,810,Рубли,24.05.2012,24.05.2022,24.05.2022,nan,nan,NaN,NaN,"700,000.00","700,000.00","203,313.61","203,313.61",0.00,nan,12.50,nan,nan,nan,nan,2,0.50,"1,016.57",0.00,0.00,0.00,1.00,"1,016.57",-92.93,"-16,263.25",0.00,0.00,-81.32,0.00,0.00,-11.61
4,2020-07-31,62837355,"45,507.00",nan,Ипотека,810,Рубли,24.05.2012,24.05.2022,24.05.2022,nan,nan,NaN,NaN,"700,000.00","700,000.00","195,087.18","195,087.18",0.00,nan,12.50,nan,nan,nan,nan,2,0.50,975.44,0.00,0.00,0.00,1.00,975.44,-41.13,"-8,226.43",0.00,0.00,-41.13,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,2020-03-31,65445195,nan,"45,815.00",Потреб,810,Рубли,05.03.2013,28.02.2018,28.02.2018,791.00,215.00,31.01.2018,30.08.2019,"1,000,000.00","1,000,000.00",0.00,nan,"20,167.94","20,167.94",17.00,nan,nan,"2,290.05","2,290.05",5,100.00,0.00,"20,167.94",0.00,"2,290.05",1.00,"22,457.99","22,457.99","20,167.94",0.00,0.00,"20,167.94",0.00,0.00,"2,290.05"
81,2020-04-30,65445195,nan,"45,815.00",Потреб,810,Рубли,05.03.2013,28.02.2018,28.02.2018,821.00,245.00,31.01.2018,30.08.2019,"1,000,000.00","1,000,000.00",0.00,nan,"20,167.94","20,167.94",17.00,nan,nan,"2,571.07","2,571.07",5,100.00,0.00,"20,167.94",0.00,"2,571.07",1.00,"22,739.01",281.02,0.00,0.00,0.00,0.00,0.00,0.00,281.02
82,2020-05-31,65445195,nan,"45,815.00",Потреб,810,Рубли,05.03.2013,28.02.2018,28.02.2018,852.00,276.00,31.01.2018,30.08.2019,"1,000,000.00","1,000,000.00",0.00,nan,"20,167.94","20,167.94",17.00,18.73,18.73,"2,842.74","2,842.74",5,100.00,0.00,"20,167.94",18.73,"2,842.74",1.00,"23,029.41",290.40,0.00,0.00,0.00,0.00,0.00,0.00,290.40
83,2020-06-30,65445195,nan,"45,815.00",Потреб,810,Рубли,05.03.2013,28.02.2018,28.02.2018,882.00,306.00,31.01.2018,30.08.2019,"1,000,000.00","1,000,000.00",0.00,nan,"20,167.94","20,167.94",17.00,nan,nan,"3,142.50","3,142.50",5,100.00,0.00,"20,167.94",0.00,"3,142.50",1.00,"23,310.44",281.03,0.00,0.00,0.00,0.00,0.00,0.00,281.03


In [11]:
cr_5[cr_5['DEAL_ID'] == 65445195]

,DT,DEAL_ID,BALMAIN,BALPRMN,PRODUCT,CURCODENUM,CURCODE,DOCBEGINDATE,ENDDATEWOPROLONG,ENDDATEWITHPROLONG,OVERDUETERMMAINDAY,OVERDUETERMPERCDAY,OVERDUEDATEMAIN,OVERDUEDATEPERC,DOCSUMCUR,DOCSUMRUR,MAINRESTCUR,MAINRESTRUR,PRMNRESTCUR,PRMNRESTRUR,DOCPROCRATE,SUMNACHPERCBALCUR,SUMNACHPERCBALRUR,SUMPRPERCBALCUR,SUMPRPERCBALRUR,RISKGROUP254,RESERVERATERPBU,SUMRESERVEMAINRPBU,SUMRESERVEPRMNRPBU,SUMRESERVEPRMN,SUMRESERVEPRPR,CUR_RATE,PROVISION,DELTA_PROVISION,dA,dC,dR,dPA,dPR,dPC,dXR
80,2020-03-31,65445195,nan,"45,815.00",Потреб,810,Рубли,05.03.2013,28.02.2018,28.02.2018,791.00,215.00,31.01.2018,30.08.2019,"1,000,000.00","1,000,000.00",0.00,nan,"20,167.94","20,167.94",17.00,nan,nan,"2,290.05","2,290.05",5,100.00,0.00,"20,167.94",0.00,"2,290.05",1.00,"22,457.99","22,457.99","20,167.94",0.00,0.00,"20,167.94",0.00,0.00,"2,290.05"
81,2020-04-30,65445195,nan,"45,815.00",Потреб,810,Рубли,05.03.2013,28.02.2018,28.02.2018,821.00,245.00,31.01.2018,30.08.2019,"1,000,000.00","1,000,000.00",0.00,nan,"20,167.94","20,167.94",17.00,nan,nan,"2,571.07","2,571.07",5,100.00,0.00,"20,167.94",0.00,"2,571.07",1.00,"22,739.01",281.02,0.00,0.00,0.00,0.00,0.00,0.00,281.02
82,2020-05-31,65445195,nan,"45,815.00",Потреб,810,Рубли,05.03.2013,28.02.2018,28.02.2018,852.00,276.00,31.01.2018,30.08.2019,"1,000,000.00","1,000,000.00",0.00,nan,"20,167.94","20,167.94",17.00,18.73,18.73,"2,842.74","2,842.74",5,100.00,0.00,"20,167.94",18.73,"2,842.74",1.00,"23,029.41",290.40,0.00,0.00,0.00,0.00,0.00,0.00,290.40
83,2020-06-30,65445195,nan,"45,815.00",Потреб,810,Рубли,05.03.2013,28.02.2018,28.02.2018,882.00,306.00,31.01.2018,30.08.2019,"1,000,000.00","1,000,000.00",0.00,nan,"20,167.94","20,167.94",17.00,nan,nan,"3,142.50","3,142.50",5,100.00,0.00,"20,167.94",0.00,"3,142.50",1.00,"23,310.44",281.03,0.00,0.00,0.00,0.00,0.00,0.00,281.03
84,2020-07-31,65445195,nan,"45,815.00",Потреб,810,Рубли,05.03.2013,28.02.2018,28.02.2018,913.00,1.00,31.01.2018,31.07.2020,"1,000,000.00","1,000,000.00",0.00,nan,"17,259.44","17,259.44",17.00,nan,nan,290.40,290.40,5,100.00,0.00,"17,259.44",0.00,290.40,1.00,"17,549.84","-5,760.60","-2,908.50",0.00,0.00,"-2,908.50",0.00,0.00,"-2,852.10"


### 6 credits 

In [12]:
cr_6 = calculate_deltas(credits_dfs[5])
credits_dfs_new.append(cr_6)
cr_6

,DT,DEAL_ID,BALMAIN,BALPRMN,PRODUCT,CURCODENUM,CURCODE,DOCBEGINDATE,ENDDATEWOPROLONG,ENDDATEWITHPROLONG,OVERDUETERMMAINDAY,OVERDUETERMPERCDAY,OVERDUEDATEMAIN,OVERDUEDATEPERC,DOCSUMCUR,DOCSUMRUR,MAINRESTCUR,MAINRESTRUR,PRMNRESTCUR,PRMNRESTRUR,DOCPROCRATE,SUMNACHPERCBALCUR,SUMNACHPERCBALRUR,SUMPRPERCBALCUR,SUMPRPERCBALRUR,RISKGROUP254,RESERVERATERPBU,SUMRESERVEMAINRPBU,SUMRESERVEPRMNRPBU,SUMRESERVEPRMN,SUMRESERVEPRPR,CUR_RATE,PROVISION,DELTA_PROVISION,dA,dC,dR,dPA,dPR,dPC,dXR
0,2020-03-31,6943416,nan,"45,815.00",Пенсионеры,810,Рубли,24.05.2011,24.05.2013,24.05.2013,"2,924.00","2,536.00",30.03.2012,22.04.2013,"41,000.00","41,000.00",0.00,nan,"21,463.40","21,463.40",0.00,nan,nan,"4,523.44","4,523.44",5,100.00,0.00,"21,463.40",0.00,"4,523.44",1.00,"25,986.84","25,986.84","21,463.40",0.00,0.00,"21,463.40",0.00,0.00,"4,523.44"
1,2020-04-30,6943416,nan,"45,815.00",Пенсионеры,810,Рубли,24.05.2011,24.05.2013,24.05.2013,"2,954.00","2,566.00",30.03.2012,22.04.2013,"41,000.00","41,000.00",0.00,nan,"21,463.40","21,463.40",0.00,nan,nan,"4,523.44","4,523.44",5,100.00,0.00,"21,463.40",0.00,"4,523.44",1.00,"25,986.84",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,2020-05-31,6943416,nan,"45,815.00",Пенсионеры,810,Рубли,24.05.2011,24.05.2013,24.05.2013,"2,985.00","2,597.00",30.03.2012,22.04.2013,"41,000.00","41,000.00",0.00,nan,"21,463.40","21,463.40",0.00,nan,nan,"4,523.44","4,523.44",5,100.00,0.00,"21,463.40",0.00,"4,523.44",1.00,"25,986.84",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,2020-06-30,6943416,nan,"45,815.00",Пенсионеры,810,Рубли,24.05.2011,24.05.2013,24.05.2013,"3,015.00","2,627.00",30.03.2012,22.04.2013,"41,000.00","41,000.00",0.00,nan,"21,463.40","21,463.40",0.00,nan,nan,"4,523.44","4,523.44",5,100.00,0.00,"21,463.40",0.00,"4,523.44",1.00,"25,986.84",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,2020-07-31,6943416,nan,"45,815.00",Пенсионеры,810,Рубли,24.05.2011,24.05.2013,24.05.2013,"3,046.00","2,658.00",30.03.2012,22.04.2013,"41,000.00","41,000.00",0.00,nan,"21,463.40","21,463.40",0.00,nan,nan,"4,523.44","4,523.44",5,100.00,0.00,"21,463.40",0.00,"4,523.44",1.00,"25,986.84",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5767,2020-04-30,63764213,nan,"45,815.00",Ипотека,978,Евро,24.01.2013,11.01.2016,11.01.2016,"1,982.00","2,160.00",27.11.2014,02.06.2014,"800,000.00","64,039,040.00",0.00,nan,"261,932.39","20,967,373.50",12.00,nan,nan,"322,425.18","25,809,748.75",5,100.00,0.00,"20,967,373.50",0.00,"25,809,748.75",80.05,"46,777,122.25","-3,104,157.02",0.00,-5.69,0.00,0.00,0.00,"-1,490,421.49","-1,613,735.53"
5768,2020-05-31,63764213,nan,"45,815.00",Ипотека,978,Евро,24.01.2013,11.01.2016,11.01.2016,"2,013.00","2,191.00",27.11.2014,02.06.2014,"800,000.00","62,839,120.00",0.00,nan,"261,932.39","20,574,501.11",12.00,"2,662.27","209,118.38","322,425.18","25,326,143.22",5,100.00,0.00,"20,574,501.11","209,118.38","25,326,143.22",78.55,"46,109,762.71","-667,359.54",0.00,-1.50,0.00,0.00,0.00,"-392,872.39","-274,487.15"
5769,2020-06-30,63764213,nan,"45,815.00",Ипотека,978,Евро,24.01.2013,11.01.2016,11.01.2016,"2,043.00","2,221.00",27.11.2014,02.06.2014,"800,000.00","62,944,960.00",0.00,nan,"261,932.39","20,609,154.76",12.00,nan,nan,"327,663.83","25,780,983.34",5,100.00,0.00,"20,609,154.76",0.00,"25,780,983.34",78.68,"46,390,138.10","280,375.39",0.00,0.13,0.00,0.00,0.00,"34,653.66","245,721.73"
5770,2020-07-31,63764213,nan,"45,815.00",Ипотека,978,Евро,24.01.2013,11.01.2016,11.01.2016,"2,074.00","2,252.00",27.11.2014,02.06.2014,"800,000.00","69,002,560.00",0.00,nan,"261,932.39","22,592,506.82",12.00,nan,nan,"330,326.09","28,491,682.31",5,100.00,0.00,"22,592,506.82",0.00,"28,491,682.31",86.25,"51,084,189.13","4,694,051.03",0.00,7.57,0.00,0.00,0.00,"1,983,352.06","2,710,698.97"


## Checking single credits for correctness

In [13]:
cr_6[cr_6['DEAL_ID'] == 62824709]

,DT,DEAL_ID,BALMAIN,BALPRMN,PRODUCT,CURCODENUM,CURCODE,DOCBEGINDATE,ENDDATEWOPROLONG,ENDDATEWITHPROLONG,OVERDUETERMMAINDAY,OVERDUETERMPERCDAY,OVERDUEDATEMAIN,OVERDUEDATEPERC,DOCSUMCUR,DOCSUMRUR,MAINRESTCUR,MAINRESTRUR,PRMNRESTCUR,PRMNRESTRUR,DOCPROCRATE,SUMNACHPERCBALCUR,SUMNACHPERCBALRUR,SUMPRPERCBALCUR,SUMPRPERCBALRUR,RISKGROUP254,RESERVERATERPBU,SUMRESERVEMAINRPBU,SUMRESERVEPRMNRPBU,SUMRESERVEPRMN,SUMRESERVEPRPR,CUR_RATE,PROVISION,DELTA_PROVISION,dA,dC,dR,dPA,dPR,dPC,dXR
1254,2020-03-31,62824709,"47,801.00",nan,Ипотека,810,Рубли,20.08.2013,20.08.2028,20.08.2028,nan,nan,NaN,NaN,"1,750,000.00","1,750,000.00","910,025.04","910,025.04",0.00,nan,12.75,nan,nan,nan,nan,2,0.35,"3,185.09",0.00,0.00,0.00,1.00,"3,185.09","3,185.09","910,025.04",0.00,0.00,"3,185.09",0.00,0.00,0.00
1255,2020-04-30,62824709,"47,801.00",nan,Ипотека,810,Рубли,20.08.2013,20.08.2028,20.08.2028,nan,nan,NaN,NaN,"1,750,000.00","1,750,000.00","904,765.55","904,765.55",0.00,nan,12.75,nan,nan,nan,nan,2,0.35,"3,166.68",0.00,0.00,0.00,1.00,"3,166.68",-18.41,"-5,259.49",0.00,0.00,-18.41,0.00,0.00,-0.00
1256,2020-05-31,62824709,"47,801.00",nan,Ипотека,810,Рубли,20.08.2013,20.08.2028,20.08.2028,nan,nan,NaN,NaN,"1,750,000.00","1,750,000.00","904,765.55","904,765.55",0.00,nan,12.75,"9,770.72","9,770.72",nan,nan,2,0.35,"3,166.68",0.00,34.20,0.00,1.00,"3,200.88",34.20,0.00,0.00,0.00,0.00,0.00,0.00,34.20
1257,2020-06-30,62824709,"47,801.00","45,815.00",Ипотека,810,Рубли,20.08.2013,20.08.2028,20.08.2028,1.00,1.00,30.06.2020,30.06.2020,"1,750,000.00","1,750,000.00","894,401.31","894,401.31","5,364.96","5,364.96",12.75,nan,nan,"8,821.50","8,821.50",2,1.50,"13,416.02",80.47,0.00,132.32,1.00,"13,628.81","10,427.93","-4,999.28",0.00,1.15,-17.50,"10,347.31",0.00,98.12
1258,2020-07-31,62824709,"47,801.00",nan,Ипотека,810,Рубли,20.08.2013,20.08.2028,20.08.2028,nan,nan,NaN,NaN,"1,750,000.00","1,750,000.00","889,290.11","889,290.11",0.00,nan,12.75,nan,nan,nan,nan,2,0.35,"3,112.52",0.00,0.00,0.00,1.00,"3,112.52","-10,516.29","-10,476.16",0.00,-1.15,-157.14,"-10,226.84",0.00,-132.31
1259,2020-08-31,62824709,"47,801.00","45,815.00",Ипотека,810,Рубли,20.08.2013,20.08.2028,20.08.2028,1.00,1.00,31.08.2020,31.08.2020,"1,750,000.00","1,750,000.00","884,123.72","884,123.72","5,166.39","5,166.39",12.75,nan,nan,"8,792.18","8,792.18",2,1.50,"13,261.86",77.50,0.00,131.88,1.00,"13,471.24","10,358.72",0.00,0.00,1.15,0.00,"10,226.84",0.00,131.88


## Combine 6 datasets into one

In [14]:
res_df = pd.DataFrame()
for i in range(len(credits_dfs_new)):
    res_df = res_df.append(credits_dfs_new[i])
res_df    
    

,DT,DEAL_ID,BALMAIN,BALPRMN,PRODUCT,CURCODENUM,CURCODE,DOCBEGINDATE,ENDDATEWOPROLONG,ENDDATEWITHPROLONG,OVERDUETERMMAINDAY,OVERDUETERMPERCDAY,OVERDUEDATEMAIN,OVERDUEDATEPERC,DOCSUMCUR,DOCSUMRUR,MAINRESTCUR,MAINRESTRUR,PRMNRESTCUR,PRMNRESTRUR,DOCPROCRATE,SUMNACHPERCBALCUR,SUMNACHPERCBALRUR,SUMPRPERCBALCUR,SUMPRPERCBALRUR,RISKGROUP254,RESERVERATERPBU,SUMRESERVEMAINRPBU,SUMRESERVEPRMNRPBU,SUMRESERVEPRMN,SUMRESERVEPRPR,CUR_RATE,PROVISION,DELTA_PROVISION,dA,dC,dR,dPA,dPR,dPC,dXR
0,2020-03-31,62944933,"47,801.00",nan,Ипотека,810,Рубли,29.05.2013,29.05.2020,29.05.2020,nan,nan,NaN,NaN,"3,000,000.00","3,000,000.00","1,621.92","1,621.92",0.00,nan,10.50,nan,nan,nan,nan,2,0.35,5.68,0.00,0.00,0.00,1.00,5.68,5.68,"1,621.92",0.00,0.00,5.68,0.00,0.00,0.00
1,2020-03-31,63049299,"47,801.00",nan,Ипотека,810,Рубли,09.11.2006,08.11.2021,08.11.2021,nan,nan,NaN,NaN,"800,000.00","800,000.00","6,292.29","6,292.29",0.00,nan,14.00,nan,nan,nan,nan,2,0.35,22.02,0.00,0.00,0.00,1.00,22.02,22.02,"6,292.29",0.00,0.00,22.02,0.00,0.00,-0.00
2,2020-03-31,63060942,nan,nan,Потреб,810,Рубли,25.12.2013,23.12.2016,23.12.2016,nan,883.00,NaN,31.10.2017,"800,000.00","800,000.00",0.00,nan,0.00,nan,0.00,nan,nan,"2,931.81","2,931.81",5,100.00,0.00,0.00,0.00,"2,931.81",1.00,"2,931.81","2,931.81",0.00,0.00,0.00,0.00,0.00,0.00,"2,931.81"
3,2020-03-31,63245372,nan,"45,815.00",Ипотека,810,Рубли,21.02.2012,21.02.2037,21.02.2037,"1,896.00","1,828.00",22.01.2015,31.03.2015,"1,560,000.00","1,560,000.00",0.00,nan,"178,791.57","178,791.57",0.00,nan,nan,"759,222.16","759,222.16",5,100.00,0.00,"178,791.57",0.00,"759,222.16",1.00,"938,013.73","938,013.73","178,791.57",0.00,0.00,"178,791.57",0.00,0.00,"759,222.16"
4,2020-03-31,63246064,"45,507.00","45,815.00",Ипотека,810,Рубли,12.03.2014,12.03.2034,12.03.2034,1.00,1.00,31.03.2020,31.03.2020,"750,000.00","750,000.00","680,533.86","680,533.86","1,325.82","1,325.82",13.00,nan,nan,"7,432.00","7,432.00",2,1.50,"10,208.01",19.89,0.00,111.48,1.00,"10,339.38","10,339.38","681,859.68",0.00,0.00,"10,227.90",0.00,0.00,111.48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5767,2020-04-30,63764213,nan,"45,815.00",Ипотека,978,Евро,24.01.2013,11.01.2016,11.01.2016,"1,982.00","2,160.00",27.11.2014,02.06.2014,"800,000.00","64,039,040.00",0.00,nan,"261,932.39","20,967,373.50",12.00,nan,nan,"322,425.18","25,809,748.75",5,100.00,0.00,"20,967,373.50",0.00,"25,809,748.75",80.05,"46,777,122.25","-3,104,157.02",0.00,-5.69,0.00,0.00,0.00,"-1,490,421.49","-1,613,735.53"
5768,2020-05-31,63764213,nan,"45,815.00",Ипотека,978,Евро,24.01.2013,11.01.2016,11.01.2016,"2,013.00","2,191.00",27.11.2014,02.06.2014,"800,000.00","62,839,120.00",0.00,nan,"261,932.39","20,574,501.11",12.00,"2,662.27","209,118.38","322,425.18","25,326,143.22",5,100.00,0.00,"20,574,501.11","209,118.38","25,326,143.22",78.55,"46,109,762.71","-667,359.54",0.00,-1.50,0.00,0.00,0.00,"-392,872.39","-274,487.15"
5769,2020-06-30,63764213,nan,"45,815.00",Ипотека,978,Евро,24.01.2013,11.01.2016,11.01.2016,"2,043.00","2,221.00",27.11.2014,02.06.2014,"800,000.00","62,944,960.00",0.00,nan,"261,932.39","20,609,154.76",12.00,nan,nan,"327,663.83","25,780,983.34",5,100.00,0.00,"20,609,154.76",0.00,"25,780,983.34",78.68,"46,390,138.10","280,375.39",0.00,0.13,0.00,0.00,0.00,"34,653.66","245,721.73"
5770,2020-07-31,63764213,nan,"45,815.00",Ипотека,978,Евро,24.01.2013,11.01.2016,11.01.2016,"2,074.00","2,252.00",27.11.2014,02.06.2014,"800,000.00","69,002,560.00",0.00,nan,"261,932.39","22,592,506.82",12.00,nan,nan,"330,326.09","28,491,682.31",5,100.00,0.00,"22,592,506.82",0.00,"28,491,682.31",86.25,"51,084,189.13","4,694,051.03",0.00,7.57,0.00,0.00,0.00,"1,983,352.06","2,710,698.97"


In [15]:
res_df['Month'] = res_df['DT'].dt.month
res_df

,DT,DEAL_ID,BALMAIN,BALPRMN,PRODUCT,CURCODENUM,CURCODE,DOCBEGINDATE,ENDDATEWOPROLONG,ENDDATEWITHPROLONG,OVERDUETERMMAINDAY,OVERDUETERMPERCDAY,OVERDUEDATEMAIN,OVERDUEDATEPERC,DOCSUMCUR,DOCSUMRUR,MAINRESTCUR,MAINRESTRUR,PRMNRESTCUR,PRMNRESTRUR,DOCPROCRATE,SUMNACHPERCBALCUR,SUMNACHPERCBALRUR,SUMPRPERCBALCUR,SUMPRPERCBALRUR,RISKGROUP254,RESERVERATERPBU,SUMRESERVEMAINRPBU,SUMRESERVEPRMNRPBU,SUMRESERVEPRMN,SUMRESERVEPRPR,CUR_RATE,PROVISION,DELTA_PROVISION,dA,dC,dR,dPA,dPR,dPC,dXR,Month
0,2020-03-31,62944933,"47,801.00",nan,Ипотека,810,Рубли,29.05.2013,29.05.2020,29.05.2020,nan,nan,NaN,NaN,"3,000,000.00","3,000,000.00","1,621.92","1,621.92",0.00,nan,10.50,nan,nan,nan,nan,2,0.35,5.68,0.00,0.00,0.00,1.00,5.68,5.68,"1,621.92",0.00,0.00,5.68,0.00,0.00,0.00,3
1,2020-03-31,63049299,"47,801.00",nan,Ипотека,810,Рубли,09.11.2006,08.11.2021,08.11.2021,nan,nan,NaN,NaN,"800,000.00","800,000.00","6,292.29","6,292.29",0.00,nan,14.00,nan,nan,nan,nan,2,0.35,22.02,0.00,0.00,0.00,1.00,22.02,22.02,"6,292.29",0.00,0.00,22.02,0.00,0.00,-0.00,3
2,2020-03-31,63060942,nan,nan,Потреб,810,Рубли,25.12.2013,23.12.2016,23.12.2016,nan,883.00,NaN,31.10.2017,"800,000.00","800,000.00",0.00,nan,0.00,nan,0.00,nan,nan,"2,931.81","2,931.81",5,100.00,0.00,0.00,0.00,"2,931.81",1.00,"2,931.81","2,931.81",0.00,0.00,0.00,0.00,0.00,0.00,"2,931.81",3
3,2020-03-31,63245372,nan,"45,815.00",Ипотека,810,Рубли,21.02.2012,21.02.2037,21.02.2037,"1,896.00","1,828.00",22.01.2015,31.03.2015,"1,560,000.00","1,560,000.00",0.00,nan,"178,791.57","178,791.57",0.00,nan,nan,"759,222.16","759,222.16",5,100.00,0.00,"178,791.57",0.00,"759,222.16",1.00,"938,013.73","938,013.73","178,791.57",0.00,0.00,"178,791.57",0.00,0.00,"759,222.16",3
4,2020-03-31,63246064,"45,507.00","45,815.00",Ипотека,810,Рубли,12.03.2014,12.03.2034,12.03.2034,1.00,1.00,31.03.2020,31.03.2020,"750,000.00","750,000.00","680,533.86","680,533.86","1,325.82","1,325.82",13.00,nan,nan,"7,432.00","7,432.00",2,1.50,"10,208.01",19.89,0.00,111.48,1.00,"10,339.38","10,339.38","681,859.68",0.00,0.00,"10,227.90",0.00,0.00,111.48,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5767,2020-04-30,63764213,nan,"45,815.00",Ипотека,978,Евро,24.01.2013,11.01.2016,11.01.2016,"1,982.00","2,160.00",27.11.2014,02.06.2014,"800,000.00","64,039,040.00",0.00,nan,"261,932.39","20,967,373.50",12.00,nan,nan,"322,425.18","25,809,748.75",5,100.00,0.00,"20,967,373.50",0.00,"25,809,748.75",80.05,"46,777,122.25","-3,104,157.02",0.00,-5.69,0.00,0.00,0.00,"-1,490,421.49","-1,613,735.53",4
5768,2020-05-31,63764213,nan,"45,815.00",Ипотека,978,Евро,24.01.2013,11.01.2016,11.01.2016,"2,013.00","2,191.00",27.11.2014,02.06.2014,"800,000.00","62,839,120.00",0.00,nan,"261,932.39","20,574,501.11",12.00,"2,662.27","209,118.38","322,425.18","25,326,143.22",5,100.00,0.00,"20,574,501.11","209,118.38","25,326,143.22",78.55,"46,109,762.71","-667,359.54",0.00,-1.50,0.00,0.00,0.00,"-392,872.39","-274,487.15",5
5769,2020-06-30,63764213,nan,"45,815.00",Ипотека,978,Евро,24.01.2013,11.01.2016,11.01.2016,"2,043.00","2,221.00",27.11.2014,02.06.2014,"800,000.00","62,944,960.00",0.00,nan,"261,932.39","20,609,154.76",12.00,nan,nan,"327,663.83","25,780,983.34",5,100.00,0.00,"20,609,154.76",0.00,"25,780,983.34",78.68,"46,390,138.10","280,375.39",0.00,0.13,0.00,0.00,0.00,"34,653.66","245,721.73",6
5770,2020-07-31,63764213,nan,"45,815.00",Ипотека,978,Евро,24.01.2013,11.01.2016,11.01.2016,"2,074.00","2,252.00",27.11.2014,02.06.2014,"800,000.00","69,002,560.00",0.00,nan,"261,932.39","22,592,506.82",12.00,nan,nan,"330,326.09","28,491,682.31",5,100.00,0.00,"22,592,506.82",0.00,"28,491,682.31",86.25,"51,084,189.13","4,694,051.03",0.00,7.57,0.00,0.00,0.00,"1,983,352.06","2,710,698.97",7


## Grouping and aggregating by date 

In [19]:
cols = ['PROVISION','DELTA_PROVISION','dA','dC','dR','dPA','dPR','dPC','dXR','Month']
agg = res_df[cols].groupby(by = ['Month']).sum()

In [21]:
agg.to_csv('Итог_месяцы.csv', index = False, sep=';', encoding='utf-8-sig')